# Discussion 3: Pandas Group Work

Now that you've tested your knowledge on the practice worksheet, it's time to actually use that knowledge on real data. We will be using the California `babynames` data from lecture.

The following are 5 tasks that are relatively complex, especially compared to what you just did on the worksheet. Form groups of 3 or 4, and choose one task to complete as a group. Discuss best strategies to accomplish the task (in terms of code efficiency, conciseness, etc.) Do not hesitate to Google for help (either to look at the Pandas documentation or StackExchange). 
 
If time permits, we will discuss our responses as a class and consider alternatives, if any. If your group finishes their task early, challenge yourself with another one!

In [1]:
import numpy as np
import pandas as pd

In [2]:
# loading up the California baby names data

import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
407423,CA,M,2022,Zayvier,5
407424,CA,M,2022,Zia,5
407425,CA,M,2022,Zora,5
407426,CA,M,2022,Zuriel,5


## Task 1

There's over 20,000 unique names in this dataset. However, some have been used for longer periods of time than others. Create a `pandas` `DataFrame` where the index is the name, and there are three columns: one corresponding to the first year in which that name appeared in the dataset, and the other corresponding to the last year in which it appeared. Add a third column corresponding to the longevity of the name (how many years it's been used), and arrange the table by decreasing longevity, and display only the first 10 names.

In [3]:
# One of many possible solutions

minmax_names = babynames.groupby('Name')['Year'].agg([min, max])
minmax_names['Longevity'] = minmax_names['max'] - minmax_names['min']
minmax_names.sort_values(by = 'Longevity', ascending = False).head(10)

,min,max,Longevity
Name,,,
Irene,1910,2022,112
Leona,1910,2022,112
Nellie,1910,2022,112
Joan,1910,2022,112
Carrie,1910,2022,112
Lila,1910,2022,112
Carolyn,1910,2022,112
Lillian,1910,2022,112
Eleanor,1910,2022,112


## Task 2

Some think that baby names are getting longer, on average, as time wears on. We're not sure if that's true, but let's check this out. Write code to return a `pandas` `Series` whose index is the year (from 1910 to 2021), and whose values are the average name length **among all babies in the dataset for that year**. *(More concretely, for each year, we are asking for the sum of the name lengths of every individual baby included in the dataset, divided by the number of babies included in that year.)*

Print out the first 10 years and the last 10 years and see if you notice any significant differences.

In [4]:
# One of many possible solutions

babynames['lengths_name'] = babynames['Name'].str.len()*babynames['Count']
avg_lens = babynames.groupby('Year')['lengths_name'].agg(sum)/babynames.groupby('Year')['Count'].agg(sum)
avg_lens.head(10)

Year
1910    5.800939
1911    5.784133
1912    5.813440
1913    5.803657
1914    5.805690
1915    5.793135
1916    5.785206
1917    5.802109
1918    5.804126
1919    5.790576
dtype: float64

In [13]:
avg_lens.tail(10)

Year
2013    5.908086
2014    5.896860
2015    5.881192
2016    5.862401
2017    5.845611
2018    5.821761
2019    5.800936
2020    5.792913
2021    5.762156
2022    5.753252
dtype: float64

## Task 3

Of the unique names given in different years, what proportion of them start with a vowel? Write `pandas` code to return a `pandas` `Series` with the index as the year and the value as the proportion of different names given that year that start with a vowel.

*Hint: the `str.startswith()` method for a Pandas series may be useful to you. You may also find it useful to define a function to plug into to the `agg` method for `groupby` objects, as in lecture.*

*Answer*: 

In [14]:
# One solution, using a user-defined function

def prop_vowels(series):
    return(np.mean(series.str.startswith(('A', 'E', 'I', 'O', 'U'))))

babynames.groupby('Year')['Name'].agg(prop_vowels)

Year
1910    0.225895
1911    0.239186
1912    0.236559
1913    0.230016
1914    0.229577
          ...   
2018    0.251412
2019    0.255978
2020    0.256938
2021    0.259508
2022    0.260289
Name: Name, Length: 113, dtype: float64

## Task 4

Are names becoming more unique over time? Return a `pandas` `Series` whose index is the `Year` and whose values are the number of names given that year whose count is less than 15. Print the first 10 and last 10 elements of the `Series`.

*Answer*:

In [15]:
# One of many possible solutions

names = babynames[babynames['Count'] < 15].groupby('Year')['Name'].count()
names.head(10)

Year
1910    205
1911    227
1912    308
1913    323
1914    384
1915    445
1916    469
1917    475
1918    513
1919    514
Name: Name, dtype: int64

In [16]:
names.tail(10)

Year
2013    3876
2014    3892
2015    3831
2016    3718
2017    3680
2018    3569
2019    3534
2020    3488
2021    3619
2022    3588
Name: Name, dtype: int64

## Task 5



Among names that were very popular (let's say, have a count greater than 1000), how many different names are there across sex and year? Write Pandas code that returns a Pandas `DataFrame`, whose columns correspond to sex and whose row indices correspond to year. Each entry should be the number of unique, "popular" names for that year for that sex (given our definition of popular from above).

If there's a `NaN` value (missing value) in the table, why do you think it's there? What do you think is a reasonable value to impute into these missing values? Fill in all missing values with the value your group finds most appropriate (look for the `fillna()` method for pandas dataframes online for info on how to do this)

*Answer: Because in 1915, there are no female names whose count is more than 1000 (see bottom cell for demonstration and code). A reasonable value to impute would be 0, since when we have a missing value, it's because there are no names that are 'popular' for that year.*

In [17]:
# One of many possible solutions

# Accomplishing first part of task, revealing NaN
df = babynames[babynames['Count'] > 1000].pivot_table(index = 'Year', columns = 'Sex', values = 'Name', aggfunc = 'count')
# Accomplishing next part of task, where we fill in 0
df.fillna(0)

Sex,F,M
Year,,
1915,0.0,1.0
1916,1.0,1.0
1917,1.0,3.0
1918,1.0,3.0
1919,1.0,2.0
...,...,...
2018,18.0,36.0
2019,18.0,37.0
2020,17.0,36.0


In [18]:
# A little justification for why we have the missing value; clear that the only 'popular' name whose count is greater than 1000 is John
# No female names were given more than 1000 times in that year

babynames[babynames['Year'] == 1915].sort_values(by = 'Count', ascending = False)

,State,Sex,Year,Name,Count,lengths_name
240686,CA,M,1915,John,1033,4132
1488,CA,F,1915,Mary,998,3992
240687,CA,M,1915,William,886,6202
240688,CA,M,1915,Robert,840,5040
1489,CA,F,1915,Dorothy,717,5019
...,...,...,...,...,...,...
1920,CA,F,1915,Madge,5,25
1919,CA,F,1915,Madelyn,5,35
1918,CA,F,1915,Lulu,5,20
1917,CA,F,1915,Lucia,5,25
